#### 梯度下降法

是一个通用的优化算法来帮助机器学习算法求解出最优解的。

所有优化算法的目的都是以最快的速度把模型参数θ求解出来

之前利用解析解公式一步求出参数的最优解，它的复杂度可以达到N的3次方，并不是常用的手段，而梯度下降法是一点点逼近最优解

<img src="./Img/6.png" width="40%" height="40%">

这个曲线其实就是损失函数，直到我们得到的MSE最小的时候，计算出来的θ就是我们的最优解

总结出来的公式：
$$
W_j^{t+1} = W_j^t - \eta * gradient_j
$$
对于$\eta$值来说，它叫学习率，必定为正，所以当梯度为正的时候，W会往小的方向调整，梯度为负的时候，W会往大的方向调整

<img src="./Img/7.png" width="50%" height="50%">   

如果损失函数是非凸函数，梯度下降法最有可能落到局部最小值的，所以其实步长不能设置太小，那样会容易落入局部最优解。

机器学习大部分损失函数是凸函数，而Deep learning 是非凸函数

#### 梯度下降法流程

梯度下降法几步流程就是猜正确答案的过程：  
* 瞎蒙，Random随机θ，随机一组数值$W_0...W_n$
* 求梯度，延切线方向往下下降就相当于沿着坡度最陡峭的方向下降
* if g < 0, θ变大，if g > 0, θ变小
* 判断是否收敛convergence，如果收敛跳出迭代，如果没有收敛，回第2步继续

<img src="./Img/8.png" width="50%" height="50%">   

怎么求梯度？  
* $gradient_j = \frac{ \partial Loss }{ \partial w_j }$  

如何调整θ或W？ 
* $W_j^{t+1} = W_j^t - \eta * gradient_j $

怎么判断收敛？
* 判断收敛这里使用g=0其实并不合理，因为当损失函数是非凸函数的话，g=0有可能是极大值点，所以我们判断loss的下降收益更合理，当随着迭代loss减少的幅度即收益不再变化就可以认为停止在最低点，收敛！

#### 三种梯度下降

区别在于第2步求梯度所用到的X的数据集的样本数量不同！它们每次学习（更新模型参数）使用的样本个数，每次更新使用不同的样本会导致每次学习的准确性和学习时间不同。

<img src="./Img/9.png" width="20%" height="20%">   

##### 1. 全量梯度下降（Batch Gradient Descent)

在梯度下降中，对于θ的更新，所有的样本都有贡献，也就是参与调整θ

一次更新的幅度是比较大的，样本不多的情况，收敛速度快

每次学习时间过长，若训练集过大，会消耗大量的内存

每次更新都会朝着正确的方向进行

##### 2. 随机梯度下降（Stochastic Gradient Descent)

每次从训练集中随机选择一个样本来学习，故每次学习的速度比较快

每次更新可能并不会按照正确的方向进行，会带来优化波动

会导致迭代次数的增加

##### 3. 小批量梯度下降（Mini-Batch Gradient Descent)

每次更新随机选择batch个样本进行学习

降低了收敛波动性，更新更稳定，选择一个更新速度和更新次数比较适合的样本数

batch的范围[50，256]